# **CrewAI agents with bedrock**

In [43]:
!pip install crewai crewai[tools] boto3 botocore sagemaker duckduckgo-search > /dev/null 2>&1

In [44]:
from crewai import LLM
from crewai_tools import SerperDevTool
llm = LLM(
    model="bedrock/us.amazon.nova-pro-v1:0",    
    temperature=0.7, max_tokens=4*1024,
)

In [45]:
MOTIVATION_AGENT_NAME = "motivation_agent"
PLANNING_AGENT_NAME = "planning_Agent"
MAIN_AGENT_NAME = "Main_Tutor_Agent"

In [52]:
from crewai import LLM, Agent, Crew, Task, Process
from textwrap import dedent
from crewai_tools import SerperDevTool #for tools

tutor_agent = Agent(
    role="Tutor Agent for student",
    goal="You are an online tutor who helps students study.",
    backstory=dedent(
        "Always greet the student, and pay attention to their tone of response to assess how they are feeling. "
        "After they study a topic and let you know, ask them how they are doing. "
        "After they take a simulation/quiz, ask them if they are feeling more confident. "
        "Try to understand the student by asking about their difficulties and goals. "
        f"If you notice the student is unmotivated, forward them to the motivation agent. "
        f"If you notice they need help planning their studies, forward them to the planning agent. "
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm
)


In [47]:
motivation_agent = Agent(
    role="Motivation Agent",
    goal="check the student's motivation level and suggest a motivational plan",
    backstory="student motivation specialist",
    verbose=True,
    llm=llm
)


In [48]:
planning_agent = Agent(
    role="Planning Agent",
    goal="create a personalized study plan",
    backstory="study plan specialist.",
    verbose=True,
    llm=llm
)

In [49]:
motivation_task = Task(
    description="Talk to the student, assess motivation and propose motivational actions",
    expected_output="Personalized motivational plan.",
    agent=motivation_agent,
)

study_plan_task = Task(
    description="Collect information, such as available study time per week, and create a personalized study plan",
    expected_output="Study plan",
    agent=planning_agent,
)

In [54]:
crew = Crew(
    agents=[motivation_agent, planning_agent],
    tasks=[motivation_task, study_plan_task],
    manager_agent=tutor_agent,

)



In [55]:
inputs = {
    "study":"Hello, I need to study math for exam next month, my level is zero, I need to become to hero. and I have 10 hours by week for study"
}
student_result = crew.kickoff(inputs=inputs)
print(student_result)

# Agent: Motivation Agent
## Task: Talk to the student, assess motivation and propose motivational actions


# Agent: Motivation Agent
## Final Answer: 
---

Hello! I'm here to help you assess your current motivation levels and provide a personalized motivational plan to help you achieve your academic goals. Let's start by discussing a few things to get a better understanding of your current situation.

### Assessment Questions:

1. **Goals:**
   - What are your short-term and long-term academic goals?
   - Are these goals specific, measurable, achievable, relevant, and time-bound (SMART)?

2. **Current Motivation Level:**
   - On a scale of 1 to 10, how would you rate your current motivation level? 
   - What factors contribute to your current motivation level?

3. **Challenges:**
   - What are the main obstacles or challenges you face in staying motivated?
   - Are there specific subjects or tasks that you find particularly demotivating?

4. **Support System:**
   - Who are the peopl

In [57]:
# Salve no histórico
history = []
history.append({"user": inputs["study"], "agent": student_result})


In [60]:
new_ask = "What is the best material to start from scratch?"

In [61]:
history_txt = ""
for turn in history:
    history_txt += f"User: {turn['user']}\nAgent: {turn['agent']}\n"

In [63]:
new_prompt = f"{history_txt}User: {new_ask}"
inputs = {"study": new_prompt}
student_result = crew.kickoff(inputs=inputs)
print(student_result)

# Agent: Motivation Agent
## Task: Talk to the student, assess motivation and propose motivational actions


# Agent: Motivation Agent
## Final Answer: 
---

Hello! I'm here to help you assess your current motivation levels and provide you with a personalized motivational plan to help you achieve your academic goals. Let's start by discussing a few things:

### Assessment of Current Motivation Levels

1. **Goals:**
   - What are your short-term and long-term academic goals?
   - How do these goals align with your personal interests and future aspirations?

2. **Current Motivation:**
   - On a scale of 1 to 10, how motivated do you feel right now?
   - What factors contribute to your current level of motivation (e.g., interest in the subject, future career goals, external pressures)?

3. **Challenges:**
   - What are the main obstacles that hinder your motivation?
   - Are there specific subjects or tasks that you find particularly demotivating?

4. **Support System:**
   - Who are the 